# DLA Abund Cookbook

In [12]:
# imports
# suppress warnings for these examples
import warnings
warnings.filterwarnings('ignore')

import json, io
import numpy as np

import astropy.units as u

from linetools.isgm.abscomponent import AbsComponent
from linetools.lists.linelist import LineList
from linetools.spectra.io import readspec
import linetools


from pyigm.abssys.dla import DLASystem

## 0.  Analyze NHI, Continuum fit the spectrum (or spectra)
   
These steps are not described here.  

For continuum fitting,  we recommend lt_continuum from linetools

    lt_continuum spec.fits

## 1.  Generate a simple DLA system

Requires $z$, $N_{\rm HI}$ and the coordinates

$v_{\rm lim}$ defaults to 500 km/s if input as None

In [7]:
# PH957
vlim = None
dla = DLASystem(('01:03:11.38','+13:16:16.7'), 2.309, vlim, 21.37)
dla

<DLASystem: 01:03:11.38 13:16:16.7, 2.309, NHI=21.37, Z/H=0>

In [8]:
# Write
dla.write_json()

Wrote J010311.38+131616.7_z2.309 system to J010311.38+131616.7_z2.309.json file


## 2. AbsKin GUI for Line IDs

   * Requires xastropy for now
   * Best run off the JSON file
   * And from the command line
   
   lt_xabssys PH957_f.fits J010311.38+131616.7_z2.309.json -outfile J010311.38+131616.7_z2.309_HIRES.json
   
   
   
   * User modifies the velocity limits
   * Identifies blends
   * Sets upper/lower limits as desired
   * Rejects lines that are not worth analysis

In [10]:
# Load
json_fil = '../../pyigm/abssys/tests/files/J010311.38+131616.7_z2.309_HIRES.json'
dla = DLASystem.from_json(json_fil)

## 3. Measure EWs and AODM columns
    

In [17]:
# Load spectrum from linetools
spec_fil = linetools.__path__[0]+'/spectra/tests/files/PH957_f.fits'
spec = readspec(spec_fil)

In [18]:
# EW
dla.measure_restew(spec=spec)

In [19]:
# AODM
dla.measure_aodm(spec=spec)

You may now wish to update the component column densities


In [20]:
# View
dla.fill_trans()
dla._trans

wrest,name,Z,ion,Ej,z,EW,sig_EW,flag_N,logN,sig_logN
Angstrom,,,,1 / cm,,Angstrom,Angstrom,,,
float64,str10,int64,int64,float64,float64,float64,float64,int64,float64,float64
1656.9284,CI 1656,6,1,0.0,2.309,0.0538278958668,0.0078618133002,1,13.2181164452,0.0586656731841
1334.5323,CII 1334,6,2,0.0,2.309,0.517674560646,0.00555575088953,2,14.7236988694,0.0158013703943
1335.7077,CII* 1335,6,2,63.42,2.309,0.296489843359,0.0059561379452,1,14.2763346029,0.00920147656081
1548.195,CIV 1548,6,4,0.0,2.309,0.0507237879034,0.00673110778867,1,13.1238546514,0.056003777332
1550.77,CIV 1550,6,4,0.0,2.309,0.0299727850519,0.00653169255047,1,13.1811857929,0.0936976117241
1200.7098,NI 1200c,7,1,0.0,2.309,0.627580928381,0.029023370279,1,15.2473276787,0.0335803532982
1238.821,NV 1238,7,5,0.0,2.309,0.892574014937,0.00794622840155,2,14.9193722766,0.0117954978639
1242.804,NV 1242,7,5,0.0,2.309,0.33355826353,0.00998233844278,1,14.5610513158,0.014221018758


## 4. Update Components

In [21]:
dla.update_component_colm()

In [22]:
dla.fill_ionN()

In [23]:
dla._ionN

Z,ion,A,Ej,z,vmin,vmax,flag_N,logN,sig_logN
,,,,,km / s,km / s,,,
int64,int64,int64,float64,float64,float64,float64,int64,float64,float64
6,1,0,0.0,2.309,-221.158657011,150.0,1,13.2181164452,0.0586656731841
6,2,0,0.0,2.309,-221.158657011,150.0,2,14.7236988694,0.0158013703943
6,2,0,63.42,2.309,-221.158657011,150.0,1,14.2763346029,0.00920147656081
6,4,0,0.0,2.309,-221.158657011,150.0,1,13.1368527717,0.0481474980317
7,1,0,0.0,2.309,-221.158657011,150.0,1,15.2473276787,0.0335803532982
7,5,0,0.0,2.309,-221.158657011,150.0,1,14.5610513158,0.014221018758
8,1,0,0.0,2.309,-221.158657011,150.0,2,15.4107237299,0.0186188356746
12,1,0,0.0,2.309,-221.158657011,150.0,1,12.7810882094,0.019012570354
